In [13]:
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [14]:
base = pd.read_csv('bases/autos.csv', encoding = 'ISO-8859-1')
base

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,2016-03-14 17:48:27,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN,2016-03-14 00:00:00,0,39576,2016-04-06 00:46:52
371524,2016-03-05 19:56:21,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein,2016-03-05 00:00:00,0,26135,2016-03-11 18:17:12
371525,2016-03-19 18:57:12,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein,2016-03-19 00:00:00,0,87439,2016-04-07 07:15:26
371526,2016-03-20 19:41:08,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN,2016-03-20 00:00:00,0,40764,2016-03-24 12:45:21


In [15]:
base = base.drop('dateCrawled', axis=1)
base = base.drop('dateCreated', axis=1)
base = base.drop('nrOfPictures', axis=1)
base = base.drop('postalCode', axis=1)
base = base.drop('lastSeen', axis=1)

In [16]:
base

,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371523,Suche_t4___vito_ab_6_sitze,privat,Angebot,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
371524,Smart_smart_leistungssteigerung_100ps,privat,Angebot,1199,test,cabrio,2000,automatik,101,fortwo,125000,3,benzin,smart,nein
371525,Volkswagen_Multivan_T4_TDI_7DC_UY2,privat,Angebot,9200,test,bus,1996,manuell,102,transporter,150000,3,diesel,volkswagen,nein
371526,VW_Golf_Kombi_1_9l_TDI,privat,Angebot,3400,test,kombi,2002,manuell,100,golf,150000,6,diesel,volkswagen,NaN


In [17]:
base['name'].value_counts()

name
Ford_Fiesta                                              657
BMW_318i                                                 627
Opel_Corsa                                               622
Volkswagen_Golf_1.4                                      603
BMW_316i                                                 523
                                                        ... 
Audi_A4_Avant_Klima_Gruene_Plakette_TÜV_&AU_NEU_XENON      1
Renault_clio_in_gold_450VB_!!                              1
Fiat_Doblo_1.6_Multijet                                    1
Renault_Laguna_1                                           1
BMW_M135i_vollausgestattet_NP_52.720____Euro               1
Name: count, Length: 233531, dtype: int64

Como há muitos muitos registros únicos em nome, e essa pode ser uma informação redundante, tendo uma vez marca, a coluna nome vai ser apagada.

In [18]:
base = base.drop('name', axis=1)

Como basicamente todos os atributos são 'privat', não vale a pena considerar seller.

In [19]:
base['seller'].value_counts()

seller
privat        371525
gewerblich         3
Name: count, dtype: int64

In [20]:
base = base.drop('seller', axis=1)

In [21]:
base['offerType'].value_counts()

offerType
Angebot    371516
Gesuch         12
Name: count, dtype: int64

In [22]:
base = base.drop('offerType', axis=1)

In [23]:
base.loc[base['price'] <= 10]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,1,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,4,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,12,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371356,0,control,NaN,2000,manuell,65,corsa,150000,0,NaN,opel,ja
371392,0,test,kleinwagen,2002,manuell,60,fiesta,150000,3,benzin,ford,NaN
371402,0,control,kleinwagen,1999,manuell,53,swift,150000,3,benzin,suzuki,NaN
371431,0,control,kleinwagen,1999,manuell,37,arosa,150000,7,benzin,seat,ja


In [24]:
base['price'].mean()

17295.14186548524

In [25]:
base = base[base['price'] > 10]

In [26]:
base.shape

(359410, 12)

In [27]:
base.loc[base['price'] > 350000]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
364171,3890000,test,coupe,2006,NaN,799,NaN,5000,7,NaN,sonstige_autos,nein
365461,599000,control,coupe,1980,manuell,377,andere,5000,3,benzin,bmw,nein
366653,99999999,control,cabrio,1996,manuell,192,3er,150000,0,NaN,bmw,NaN
366861,3895000,test,coupe,2006,NaN,799,NaN,5000,4,benzin,sonstige_autos,nein


In [28]:
base = base[base['price'] < 350000]

In [29]:
base.shape

(359291, 12)

In [30]:
base[pd.isnull(base['vehicleType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [31]:
base['vehicleType'].value_counts()

vehicleType
limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: count, dtype: int64

In [32]:
base['vehicleType'].mode()

0    limousine
Name: vehicleType, dtype: object

In [33]:
base[pd.isnull(base['gearbox'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371443,3300,control,kombi,2006,NaN,0,touran,150000,7,diesel,volkswagen,NaN
371460,3500,control,NaN,1995,NaN,0,polo,150000,0,NaN,volkswagen,NaN
371486,350,control,kleinwagen,1996,NaN,65,punto,150000,0,NaN,fiat,NaN
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN


In [34]:
base['gearbox'].value_counts()

gearbox
manuell      266547
automatik     75508
Name: count, dtype: int64

In [35]:
base[pd.isnull(base['model'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein
...,...,...,...,...,...,...,...,...,...,...,...,...
371399,560,control,kleinwagen,2001,automatik,170,NaN,90000,0,benzin,fiat,ja
371476,9400,control,kombi,2007,manuell,200,NaN,150000,4,diesel,sonstige_autos,ja
371495,180,control,NaN,1995,NaN,0,NaN,125000,3,benzin,opel,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein


In [36]:
base['model'].value_counts()

model
golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: count, Length: 251, dtype: int64

In [37]:
base[pd.isnull(base['fuelType'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371496,3850,test,cabrio,2006,manuell,108,2_reihe,125000,2,NaN,peugeot,nein
371504,2600,control,NaN,2005,automatik,0,c_klasse,150000,9,NaN,mercedes_benz,NaN
371509,1900,test,NaN,2000,manuell,110,NaN,150000,7,NaN,volkswagen,nein
371519,5250,control,NaN,2016,automatik,150,159,150000,12,NaN,alfa_romeo,nein


In [38]:
base['fuelType'].value_counts()

fuelType
benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: count, dtype: int64

In [39]:
base[pd.isnull(base['notRepairedDamage'])]

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
371507,5999,test,kombi,2005,manuell,140,a4,150000,4,diesel,audi,NaN
371514,999,control,cabrio,2000,manuell,95,megane,150000,4,benzin,renault,NaN
371515,1690,test,kombi,2004,manuell,55,fabia,150000,4,benzin,skoda,NaN
371523,2200,test,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN


In [40]:
base['notRepairedDamage'].value_counts()

notRepairedDamage
nein    259301
ja       34004
Name: count, dtype: int64

In [41]:
values = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'notRepairedDamage': 'nein',
    'fuelType': 'benzin',
    'model': 'golf',
}
base = base.fillna(value = values)

In [42]:
base.isnull().sum()

price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
kilometer              0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dtype: int64

In [43]:
x = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [44]:
x

array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
       ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
       ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
       ...,
       ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
       ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
       ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
      dtype=object)

In [45]:
y

array([  480, 18300,  9800, ...,  9200,  3400, 28990])

In [46]:
base['brand'].value_counts()

brand
volkswagen        76755
bmw               38982
opel              38557
mercedes_benz     34498
audi              31884
ford              24761
renault           17367
peugeot           10777
fiat               9339
seat               6815
skoda              5572
mazda              5529
smart              5159
citroen            5038
nissan             4893
toyota             4616
hyundai            3585
mini               3354
sonstige_autos     3331
volvo              3229
mitsubishi         2956
honda              2726
kia                2494
suzuki             2274
alfa_romeo         2250
porsche            2154
chevrolet          1791
chrysler           1373
dacia               889
jeep                784
daihatsu            774
land_rover          759
subaru              746
jaguar              603
daewoo              530
saab                516
trabant             489
lancia              467
rover               462
lada                213
Name: count, dtype: int64

In [47]:
base.columns

Index(['price', 'abtest', 'vehicleType', 'yearOfRegistration', 'gearbox',
       'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType',
       'brand', 'notRepairedDamage'],
      dtype='object')

In [48]:
ohe = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder='passthrough')

In [49]:
x = ohe.fit_transform(x).toarray()
x

array([[0.00e+00, 1.00e+00, 0.00e+00, ..., 0.00e+00, 1.50e+05, 0.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.90e+02, 1.25e+05, 5.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.63e+02, 1.25e+05, 8.00e+00],
       ...,
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.02e+02, 1.50e+05, 3.00e+00],
       [0.00e+00, 1.00e+00, 0.00e+00, ..., 1.00e+02, 1.50e+05, 6.00e+00],
       [1.00e+00, 0.00e+00, 0.00e+00, ..., 3.20e+02, 5.00e+04, 8.00e+00]])

In [42]:
x.shape

(359291, 316)

## **Rede Neural**

In [17]:
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.models import Sequential

In [45]:
# mean(entradas, saidas)
(316 + 1)/2

158.5

In [47]:
regressor = Sequential([
    InputLayer(shape=(316,)),
    Dense(units=(159), activation='relu'),
    Dense(units=(159), activation='relu'),
    Dense(units=1, activation='linear'),
])

In [48]:
regressor.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 159)            │        50,403 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 159)            │        25,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,003 (296.89 KB)

 Trainable params: 76,003 (296.89 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [50]:
regressor.fit(x, y, batch_size=300, epochs=100)

2024-09-06 11:25:42.635631: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 454143824 exceeds 10% of free system memory.


Epoch 1/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 4162.2466 - mean_absolute_error: 4162.2466
Epoch 2/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - loss: 3436.6816 - mean_absolute_error: 3436.6816
Epoch 3/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 3059.2632 - mean_absolute_error: 3059.2632
Epoch 4/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 2895.1519 - mean_absolute_error: 2895.1521
Epoch 5/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2800.2983 - mean_absolute_error: 2800.2983
Epoch 6/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 2744.2751 - mean_absolute_error: 2744.2754
Epoch 7/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 2690.7185 - mean_absolute_error: 2690.7185
Epoch 8/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - loss: 2651.4653 - mean_absolute_error: 2651.4653
Epoch 9/100
1198/1198 ━━━━━━━━━━━━━━━━━━━━ 13s 11ms/step - loss: 2611.2065 - mean_absolute_error: 2611.2065
Epoch 10/100
1198/1198 ━━━━━━━━━━━

In [53]:
regressor.save('./models/regressor.keras')

In [18]:
from tensorflow.keras.saving import load_model
regressor = load_model('models/regressor.keras')

In [20]:
prevs = regressor.predict(x)

2024-09-06 13:35:45.178868: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 454143824 exceeds 10% of free system memory.


11228/11228 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step


In [21]:
y.mean(), prevs.mean()

(5916.833945186492, 5984.5073)

## **Cross Validation**

In [23]:
import time
from tensorflow.keras import backend as k
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score

In [28]:
inicio = time.time()

In [26]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        InputLayer(shape=(316,)),
        Dense(units=(159), activation='relu'),
        Dense(units=(159), activation='relu'),
        Dense(units=1, activation='linear'),
    ])
    regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])
    return regressor

In [29]:
rn = KerasRegressor(model=criar_rede, epochs=100, batch_size=300)
results = cross_val_score(estimator=rn, X=x, y=y, cv=5, scoring='neg_mean_absolute_error')

2024-09-06 14:14:33.958895: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363314048 exceeds 10% of free system memory.


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - loss: 4291.0059 - mean_absolute_error: 4291.0059
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 40s 26ms/step - loss: 3500.7239 - mean_absolute_error: 3500.7239
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 27s 28ms/step - loss: 3423.4641 - mean_absolute_error: 3423.4644
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 3361.3977 - mean_absolute_error: 3361.3977
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 13s 14ms/step - loss: 3198.7844 - mean_absolute_error: 3198.7844
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step - loss: 2926.3657 - mean_absolute_error: 2926.3657
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 2813.6045 - mean_absolute_error: 2813.6045
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - loss: 2744.2456 - mean_absolute_error: 2744.2456
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 2708.2327 - mean_absolute_error: 2708.2327
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms

2024-09-06 14:39:05.155695: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90829776 exceeds 10% of free system memory.


240/240 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


2024-09-06 14:39:45.883973: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363315312 exceeds 10% of free system memory.


Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 22s 19ms/step - loss: 4320.7988 - mean_absolute_error: 4320.7988
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - loss: 3453.7827 - mean_absolute_error: 3453.7827
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 26s 27ms/step - loss: 3200.9875 - mean_absolute_error: 3200.9878
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 37s 22ms/step - loss: 2990.1851 - mean_absolute_error: 2990.1851
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 33s 34ms/step - loss: 2894.9246 - mean_absolute_error: 2894.9246
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 2837.3628 - mean_absolute_error: 2837.3630
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 2775.4026 - mean_absolute_error: 2775.4026
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 2713.6660 - mean_absolute_error: 2713.6660
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 2702.0554 - mean_absolute_error: 2702.0554
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/ste

In [30]:
fim = time.time()

In [32]:
((fim - inicio) / 60) / 60

2.016736271646288

In [33]:
abs(results)

array([2281.54250675, 2510.60887474, 2204.18513374, 2668.69557717,
       2347.71082609])

In [34]:
abs(results.mean())

2402.548583695384

In [6]:
import tensorflow.keras.backend as k

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [4]:
def criar_rede_tuning(loss):
    k.clear_session()
    regressor = Sequential([
        InputLayer(shape=(316,)),
        Dense(units=(159), activation='relu'),
        Dense(units=(159), activation='relu'),
        Dense(units=1, activation='linear'),
    ])
    regressor.compile(optimizer='adam', loss=loss, metrics=['mean_absolute_error'])
    return regressor

regressor = KerasRegressor(model=criar_rede_tuning, verbose=0)

In [5]:
params = {
    'model__loss': [
        'mean_squared_error' , 
        'mean_absolute_error' , 
        'mean_absolute_percentage_error' , 
        'mean_squared_logarithmic_error', 
        'squared_hinge'
    ]
}

In [50]:
gs = GridSearchCV(estimator=regressor, param_grid=params, cv=5, scoring='neg_mean_absolute_error', verbose=3)
gs.fit(x, y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


2024-09-09 12:57:26.654295: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363314048 exceeds 10% of free system memory.
2024-09-09 12:57:53.211275: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90829776 exceeds 10% of free system memory.


[CV 1/5] END model__loss=mean_squared_error;, score=-3061.072 total time=  38.4s


2024-09-09 12:58:00.486254: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363315312 exceeds 10% of free system memory.
2024-09-09 12:58:27.821992: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90828512 exceeds 10% of free system memory.


[CV 2/5] END model__loss=mean_squared_error;, score=-3148.445 total time=  34.7s


2024-09-09 12:58:34.676444: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 363315312 exceeds 10% of free system memory.


[CV 3/5] END model__loss=mean_squared_error;, score=-2903.125 total time=  32.4s
[CV 4/5] END model__loss=mean_squared_error;, score=-3376.616 total time=  31.9s
[CV 5/5] END model__loss=mean_squared_error;, score=-3689.738 total time=  31.9s
[CV 1/5] END model__loss=mean_absolute_error;, score=-3263.267 total time=  32.8s
[CV 2/5] END model__loss=mean_absolute_error;, score=-2990.211 total time=  33.9s
[CV 3/5] END model__loss=mean_absolute_error;, score=-2936.278 total time=  31.7s
[CV 4/5] END model__loss=mean_absolute_error;, score=-2862.615 total time=  31.6s
[CV 5/5] END model__loss=mean_absolute_error;, score=-2789.675 total time=  31.9s
[CV 1/5] END model__loss=mean_absolute_percentage_error;, score=-5210.933 total time=  32.9s
[CV 2/5] END model__loss=mean_absolute_percentage_error;, score=-5053.499 total time=  32.4s
[CV 3/5] END model__loss=mean_absolute_percentage_error;, score=-5239.341 total time=  32.6s
[CV 4/5] END model__loss=mean_absolute_percentage_error;, score=-506

GridSearchCV(cv=5,
             estimator=KerasRegressor(model=<function criar_rede_tuning at 0x78c31735c430>, verbose=0),
             param_grid={'model__loss': ['mean_squared_error',
                                         'mean_absolute_error',
                                         'mean_absolute_percentage_error',
                                         'mean_squared_logarithmic_error',
                                         'squared_hinge']},
             scoring='neg_mean_absolute_error', verbose=3)

In [52]:
abs(gs.best_score_)

2968.4091191997995

In [54]:
gs.best_params_

{'model__loss': 'mean_absolute_error'}